# Import Dependencies

In [69]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
import numpy as np
import cv2

from ChannelFeatures import ChannelFeatures
from feature_generator import FeatureGenerator
import nms

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Detector Class

In [39]:
class Detector():
    """ 
        The Detector class is used to detect pedestrians in images by locating bounding boxes with high probabilities
        containing a pedestrian 
    """

    def __init__(self, clf, fg, window_size=(120,60), scaling_factor=1.2, scaling_iters=3, window_step=24):
        """ Instantiates the detector class:
        
            Input: weight_indices, weights, window_size, scaling_factor, scaling_iters, window_step
            
            - weight_indices: the indices of the features that will be used to score a window in the image
            - weights: the weights used to compute a score for a feature vector associated with a window in the image
            - fg: FeatureGenerator() object used to generate feature vectors for window in an image
            - window_size: the size of the sliding window defaults to 120 x 60
            - scaling_factor: factor by which we scale the image on each successive scaling iteration
            - scaling_iters: the number of times we scale the image
            - window_step: the amount of pixels stepped over on each slide of the window
        """

        self.clf = clf
        self.window_size = window_size
        self.window_step = window_step

        self.scaling_factor = scaling_factor
        self.scaling_iters = scaling_iters

        self.cf = ChannelFeatures()
        self.fg = fg

    def detect_pedestrians(self, img_path):
        """
            Detects pedestrians in an image.

            1) Slides bounding box window over the image
            2) Computes detection score using weights from boosted tree classifier
            3) Keeps the bounding box if the score is above a certain threshold
            4) Runs non-maximal suppression (NMS) on bounding boxes

            Input: img_path

            - img_path: path to image file

            Output: list of bounding boxes and scores 

        """
        
        candidate_bbs = self._get_bounding_boxes(img_path)
        bbs = nms.non_max_suppresion(candidate_bbs, overlapThresh=0.5)
        return bbs
        

       
        
    def _get_bounding_boxes(self, img_path):
        """ 
            Returns 2D array of bounding boxes (M bounding boxes x 5 characteristics per bounding box)
        """
        
        bounding_boxes = []

        img = cv2.imread(img_path)

        oheight, owidth, channels = img.shape
        win_h, win_w = self.window_size
        
        total_iters = self._calculate_total_iters(img)
        count = 0
        print "%d Total Iterations" % (total_iters)
        
        #=====[ Collect bounding boxes for each scaling iteration ]=====
        for it_num in range(self.scaling_iters):

            #=====[ Scale image if not on first iteration ]=====
            if it_num > 0:
                img = cv2.resize(img,(int(it_num*self.scaling_factor*owidth), int(it_num*self.scaling_factor*oheight)))

            height, width, _ = img.shape

            y_range = (height - win_h)/self.window_step + 1
            x_range = (width - win_w)/self.window_step + 1
            
            cfeats = self.cf.compute_channels(img)

            #=====[ Slide window across entirety of image and calculate bounding box at each step ]=====
            for y in range(y_range):
                for x in range(x_range):

                    y_pix = y*self.window_step
                    x_pix = x*self.window_step

                    #=====[ Score the bounding box ]=====
                    feature_vec = np.asarray(self.fg.generate_features(cfeats[y:y+win_h,x:x+win_w]))
                    score = self.clf.predict(feature_vec)

                    #=====[ Scale and store bounding box ]=====
                    scale = self.scaling_factor*it_num if it_num else 1
                    bounding_boxes.append([score, y_pix/scale, x_pix/scale, win_h/scale, win_w/scale])


            return np.matrix(bounding_boxes)
        
    def _calculate_total_iters(self, img):
        """ Calculates total number of bounding box scores to be calculated """
        
        oheight, owidth, channels = img.shape
        win_h, win_w = self.window_size
        
        iters = 0
        
        for it_num in range(self.scaling_iters):

            #=====[ Scale image if not on first iteration ]=====
            if it_num > 0:
                img = cv2.resize(img,(int(it_num*self.scaling_factor*owidth), int(it_num*self.scaling_factor*oheight)))

            height, width, _ = img.shape

            y_range = (height - win_h)/self.window_step + 1
            x_range = (width - win_w)/self.window_step + 1
            
            iters += y_range*x_range
        
        return iters

In [43]:
from template_generator import TemplateGenerator
import pickle
import sys
sys.path.append('../')

print "BB Detection for full feature set classifier is about 7 minutes"
print "BB Detection for small feature set classifier is about 2 minutes"

clf = pickle.load(open('../BoostedTreeclassifier_small.p','rb'))
print "%d Estimators Used" % (len(clf.clf.estimator_weights_))
print "%d Features" % (len(clf.clf.feature_importances_))

tg = TemplateGenerator()
tg.generate_sizes(w_max=3, h_max=2)
# tg.generate_sizes()
templates = tg.generate_templates()

#=====[ Instantiate FeatureGenerator ]=====
fg = FeatureGenerator(templates)

d = Detector(clf.clf,fg)

2000 Estimators Used
24926 Features
Created 2266 templates


In [44]:
import datetime
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

print datetime.datetime.now()
bb_large = d.detect_pedestrians('../image.png')
print datetime.datetime.now()

2016-05-31 15:22:31.822488
12015 Total Iterations
2016-05-31 15:29:34.049866


In [45]:
print len(bb)
print len(bb_large)

1334
1334


In [70]:
print datetime.datetime.now()
nms_bb = nms.non_max_suppression(bb, overlapThresh=0.5)
print datetime.datetime.now()

2016-05-31 15:35:27.022889


NameError: global name 'x1' is not defined

In [67]:
h = bb[:, 3]
w = bb[:, 4]
len(h)

1334

In [68]:
len(np.multiply(h,w))

1334